In [1]:

%pprint
%matplotlib inline
import sys
import os.path as osp

executable_path = sys.executable; scripts_folder = osp.join(osp.dirname(executable_path), 'Scripts')
py_folder = osp.abspath(osp.join('..', 'py')); ffmpeg_folder = r'C:\ffmpeg\bin'
if (scripts_folder not in sys.path): sys.path.insert(1, scripts_folder)
if (py_folder not in sys.path): sys.path.insert(1, py_folder)
if (ffmpeg_folder not in sys.path): sys.path.insert(1, ffmpeg_folder)
from jobpostlib import (crf, cu, datetime, duration, hau, hc, humanize, ihu, lru, nu, osp, scrfcu, slrcu, ssgdcu, su, t0, time, wsu, speech_engine)
from pandas import DataFrame
import re
import pyperclip
import ipywidgets as widgets
from IPython.display import display

Pretty printing has been turned OFF
======== Neo4j/5.24.2 ========
Utility libraries created in 4 seconds



## Get some example job posting HTML

In [6]:

# Get the most unbalance POS symbol
min_recall_pos_symbol = nu.load_object('min_recall_pos_symbol')

# Get all summarized child strings
cypher_str = f'''
    // Filter for NavigableParents nodes with an unambiguous SUMMARIZES relationship
    MATCH (np:NavigableParents)
    WHERE size([(np)<-[:SUMMARIZES]-(:PartsOfSpeech) | 1]) = 1
    
    // Get the child strings summarized by {min_recall_pos_symbol}
    WITH np
    MATCH (pos:PartsOfSpeech)-[r:SUMMARIZES]->(np:NavigableParents)
    WHERE pos.pos_symbol = "{min_recall_pos_symbol}"
    
    RETURN DISTINCT np.navigable_parent AS navigable_parent;'''#; print(cypher_str)
row_objs_list = []
with cu.driver.session() as session: row_objs_list = session.write_transaction(cu.do_cypher_tx, cypher_str)
if row_objs_list:
    target_class_df = DataFrame(row_objs_list)
    base_headers = target_class_df.navigable_parent.to_list()
    if target_class_df.shape[0]:
        print(target_class_df.shape)
        display(target_class_df.head(5))

(106, 1)


,navigable_parent
0,<b>Application Process</b>
1,<b>Apply here:</b>
2,<b>Click Here to apply directly :</b>
3,<b>How to apply?</b>
4,<b>Link to apply:</b>



## Generate synthetic job posting HTML

In [7]:

print(f'''
element_strs_set = set({sorted(base_headers)})''')


element_strs_set = set([', Senior Recruiter Stan Blackwell (sblackwell@delmock.com / 410-218-0900) would love to speak with you regarding the following position:', '<b>ALL APPLICANTS:</b>', '<b>About this opportunity:</b>', '<b>Application Materials Required</b>', '<b>Application Process</b>', '<b>Apply here:</b>', '<b>Background Check</b>', '<b>CTAP/ICTAP candidates:</b>', '<b>Click Here to apply directly :</b>', '<b>Entry-level:</b>', '<b>How to Apply:</b>', '<b>How to Apply</b>', '<b>How to apply?</b>', '<b>If specific educational requirements are indicated for this vacancy:</b>', '<b>If this opportunity interests you, you might like these courses on Coursera:</b>', '<b>Interview Procedure:</b>', '<b>Interview Procedures:</b>', '<b>Interview Process:</b>', '<b>Key words to search for the candidate</b>', '<b>LINKS</b>', '<b>Learn more by visiting:</b>', '<b>Link to apply:</b>', '<b>Mid-level:</b>', '<b>OPM Qualification Standards for the GS-1530 series can be found at the following 

In [8]:

import random

element_strs_set = set([
    ', Senior Recruiter Stan Blackwell (sblackwell@delmock.com / 410-218-0900) would love to speak with you regarding the following position:', '<b>ALL APPLICANTS:</b>',
    '<b>About this opportunity:</b>', '<b>Application Materials Required</b>', '<b>Application Process</b>', '<b>Apply here:</b>', '<b>Background Check</b>',
    '<b>CTAP/ICTAP candidates:</b>', '<b>Click Here to apply directly :</b>', '<b>Entry-level:</b>', '<b>How to Apply:</b>', '<b>How to Apply</b>', '<b>How to apply?</b>',
    '<b>If specific educational requirements are indicated for this vacancy:</b>', '<b>If this opportunity interests you, you might like these courses on Coursera:</b>',
    '<b>Interview Procedure:</b>', '<b>Interview Procedures:</b>', '<b>Interview Process:</b>', '<b>Key words to search for the candidate</b>', '<b>LINKS</b>',
    '<b>Learn more by visiting:</b>', '<b>Link to apply:</b>', '<b>Mid-level:</b>',
    '<b>OPM Qualification Standards for the GS-1530 series can be found at the following website:</b>', '<b>Onboarding duration –</b>', '<b>Our recruitment process:</b>',
    '<b>Persons with Disabilities:</b>', '<b>Please be prepared to answer the following questions before the interview:</b>', '<b>Please submit the following documents:</b>',
    '<b>Posting Contact Email</b>', '<b>Posting Contact Name</b>', '<b>Pre-requisite checklist/expectations from the candidate before sharing the relevant profile:</b>',
    '<b>Questions that need answered</b>', '<b>Referral</b>', '<b>Required Documents:</b>', '<b>STEP 1:</b>', '<b>STEP 2:</b>', '<b>STEP 3:</b>',
    '<b>Screening of Applications Begins:</b>', '<b>Senior-level:</b>', '<b>To Know More:</b>', '<b>To all recruitment agencies:</b>', '<b>Veterans:</b>',
    '<b>Want more info?</b>', "<b>Want to learn more about Berkeley Lab's Culture, Benefits and answers to FAQs? Please visit:</b>",
    '<b>We need participants on the following topics:</b>', '<b>What Else You Need to Know</b>', '<b>What happens after you apply?</b>', '<b>What is our hiring process?</b>',
    '<b>What to expect in the hiring process:</b>', '<b>Work Authorization/Sponsorship</b>', '<div>Applicant Notes:</div>',
    '<div>Required application materials (preferably in PDF Format):</div>', '<h1>How We Interview</h1>', '<h2 class="css-14vqcyj e1tiznh50">Hiring Insights</h2>',
    '<h2 class="js-match-insights-provider-14dlqhn e1tiznh50">Profile insights</h2>', '<h3 class="jobSectionHeader" pos="H-IP">Career roadmap after the bootcamp?:</h3>',
    '<h3 class="jobSectionHeader" pos="H-IP">Guides:</h3>', '<h3 class="jobSectionHeader" pos="H-IP">Interview Procedures:</h3>',
    '<h3 class="jobSectionHeader">Interview Procedures:</h3>',
    '<h3 class="js-match-insights-provider-11n8e9a e1tiznh50">Licenses</h3>', '<h3 class="text-white font-semibold text-lg mb-2">How is this Skills Bootcamp delivered?</h3>',
    '<h3 class="text-white font-semibold text-lg mb-2">Programme schedule</h3>',
    '<h3 class="text-white font-semibold text-lg mb-2">What you\'ll learn in this Skills Bootcamp</h3>', '<li>Cover letter including:</li>',
    '<li>Experience - multiple levels can be accommodated:</li>', '<li>Please read the following guidance.</li>', '<p>2. Other supporting documents:</p>',
    '<p>Application Question(s):</p>', '<p>COVID-19 Precaution(s):</p>', '<p>Experience level:</p>', '<p>For full consideration, please submit the following document(s):</p>',
    '<p>HOW TO APPLY</p>', "<p>Here's what you can expect from our hiring process:</p>",
    '<p>The selection process consists of four steps:</p>', "<p>We'd love to talk to you! Please submit the following to apply:</p>", '<p>What to Expect Next</p>',
    '<span style="font-size:11.0pt">If you’re interested, please provide below details:</span>', '<span style="font-size:12.0pt">Interview Process</span>',
    '<strong>Interview process:</strong>', '<strong>Next steps:</strong>', '<strong>Safety and respectful learning environment notice:</strong>', '<strong>To apply:</strong>',
    '<strong>Want to learn more?</strong>', "<strong>What's Next:</strong>", 'Application Notes:', 'Application response rate:', 'COVID-19 Considerations:', 'CRITICAL NOTES:',
    'Date and times available to interview:', 'Find out more:', 'For more relevant job opportunities please visit our website:', 'Full Name:', 'HIRING SPECIFICS', 'Report job',
    'Some things we think you should know', 'Stay connected',
    'The scored occupational questionnaire will evaluate you on the following competencies; please do not provide a separate written response:', 'Visa Sponsorship Information:',
    'Work Arrangement', 'You will be rated on the following Competencies as part of the assessment questionnaire for this position:'
])
base_headers = random.sample(element_strs_set, 10)

In [9]:

import cohere

# Initialize the Cohere client
co_key = wsu.secrets_json['Cohere_API_Key']
co = cohere.Client(co_key)

# Define the base headers for Post Date Headers
prompt = f"Output a python list (not a function that generates a list) of job posting HTML examples similar to these {hc.POS_EXPLANATION_DICT[min_recall_pos_symbol]}s ({min_recall_pos_symbol}):\n" + str(base_headers)
print(prompt)

Output a python list (not a function that generates a list) of job posting HTML examples similar to these Interview Procedures Headers (H-IP):
["<p>We'd love to talk to you! Please submit the following to apply:</p>", 'Date and times available to interview:', '<p>2. Other supporting documents:</p>', '<h2 class="js-match-insights-provider-14dlqhn e1tiznh50">Profile insights</h2>', '<strong>Interview process:</strong>', '<b>About this opportunity:</b>', '<h1>How We Interview</h1>', 'Some things we think you should know', '<h3 class="js-match-insights-provider-11n8e9a e1tiznh50">Licenses</h3>', '<b>Persons with Disabilities:</b>']


In [10]:

# Generate synthetic headers
response = co.generate(
    prompt=prompt,
    model='command-xlarge-nightly',
    num_generations=5,
    max_tokens=200,
    temperature=0.7,
)
generation_texts_list = [cohere_generation_obj.text for cohere_generation_obj in response.generations]
print(max(generation_texts_list, key=lambda x: len(str(x))))

unknown field: parameter model is not a valid field


Here is a Python list of job posting HTML examples similar to the provided Interview Procedures Headers:
```python
[
    "<p>To apply, please send your resume and a brief introduction to [email protected> by Friday, May 10th.</p>",
    '<h2>Interview Process Overview</h2>',
    "<p>1. Initial screening call with HR</p>",
    '<h4>Our Interview Process</h4>',
    '<strong>What to Expect:</strong>',
    '<p>We will be conducting interviews on a rolling basis, so we encourage early applications.</p>',
    '<h3>Interview Stages</h3>',
    '<b>Step 1: Online Application</b>',
    "<p>We're excited to hear from you! Here's what you can expect during the interview process:</p>",
    '<h1>Interview Tips</h1>',
    '<p>3. Shortlisted candidates will be invited for a virtual interview


In [11]:

for generation_text in generation_texts_list:
    # display(generation_text)
    element_strs_list = re.findall(r"""['`]([^\\'`\]\[]+)['`],?""", generation_text)
    for element_str in element_strs_list:
        html_str = element_str.strip('\'`').strip()
        if html_str.startswith('<') or html_str.endswith('>'):
            html_str = hau.get_navigable_children(hau.get_body_soup(html_str), [])[0].strip()

            # Disqualify text surrounded by inline elements
            if not any(map(lambda pe: html_str.endswith(f'</{pe}>'), cu.inline_elements_set)):
                element_strs_set.add(html_str)
        else:
            element_strs_set.add(html_str)

In [ ]:

# Check if the slrcu has built its parts-of-speech logistic regression model
# Parts-of-speech logistic regression model is normally built in 1 hour, 10 minutes and 4 seconds
t1 = time.time()
if not hasattr(slrcu, 'pos_predict_percent_fit_dict'):
    slrcu.build_pos_logistic_regression_elements(sampling_strategy_limit=None, verbose=True)
if hasattr(slrcu, 'pos_predict_percent_fit_dict'): print('predict_single is available')
else: print('predict_single is not available')
duration_str = humanize.precisedelta(time.time() - t1, minimum_unit='seconds', format='%0.0f')
speech_str = f'Parts-of-speech logistic regression model built in {duration_str}'; print(speech_str)

# Display the HTML that the classifier thinks looks good
db_pos_list = []
child_strs_list = sorted(element_strs_set)
for navigable_parent in child_strs_list:
    db_pos_list = cu.append_parts_of_speech_list(navigable_parent, pos_list=db_pos_list, verbose=False)
for navigable_parent, db_pos_symbol in zip(child_strs_list, db_pos_list):
    if db_pos_symbol and ('-' in db_pos_symbol):
        # print(f'*{db_pos_symbol}: {navigable_parent}')
        continue
    else:
        assert hasattr(slrcu, 'pos_predict_percent_fit_dict'), 'slrcu.predict_single needs to be available'
        pr_pos_symbol = slrcu.predict_single(navigable_parent)
        if pr_pos_symbol == min_recall_pos_symbol:
            print(f'{pr_pos_symbol}: {navigable_parent}')


## Add (pos:PartsOfSpeech)-[r:SUMMARIZES]->(np:NavigableParents) pairs to the database

In [12]:

print(f'''
child_strs_list = {sorted(element_strs_set)}''')


child_strs_list = [',', ',\n    "<p>2. Technical Assessment: A practical test to evaluate your skills.</p>",', ',\n    "<p>Let', ', Senior Recruiter Stan Blackwell (sblackwell@delmock.com / 410-218-0900) would love to speak with you regarding the following position:', '<b>ALL APPLICANTS:</b>', '<b>About this opportunity:</b>', '<b>Application Materials Required</b>', '<b>Application Process</b>', '<b>Apply here:</b>', '<b>Background Check</b>', '<b>CTAP/ICTAP candidates:</b>', '<b>Click Here to apply directly :</b>', '<b>Entry-level:</b>', '<b>How to Apply:</b>', '<b>How to Apply</b>', '<b>How to apply?</b>', '<b>If specific educational requirements are indicated for this vacancy:</b>', '<b>If this opportunity interests you, you might like these courses on Coursera:</b>', '<b>Interview Procedure:</b>', '<b>Interview Procedures:</b>', '<b>Interview Process:</b>', '<b>Key words to search for the candidate</b>', '<b>LINKS</b>', '<b>Learn more by visiting:</b>', '<b>Link to apply:</b>', '

In [13]:

# Take these hand-picked HTML strings and add them to the database
from IPython.display import clear_output

child_strs_list = [
    '<p>2. Technical Assessment: A practical test to evaluate your skills.</p>',
    ', Senior Recruiter Stan Blackwell (sblackwell@delmock.com / 410-218-0900) would love to speak with you regarding the following position:', '<b>ALL APPLICANTS:</b>',
    '<b>About this opportunity:</b>', '<b>Application Materials Required</b>', '<b>Application Process</b>', '<b>Apply here:</b>', '<b>Background Check</b>',
    '<b>CTAP/ICTAP candidates:</b>', '<b>Click Here to apply directly :</b>', '<b>Entry-level:</b>', '<b>How to Apply:</b>', '<b>How to Apply</b>', '<b>How to apply?</b>',
    '<b>If specific educational requirements are indicated for this vacancy:</b>', '<b>If this opportunity interests you, you might like these courses on Coursera:</b>',
    '<b>Interview Procedure:</b>', '<b>Interview Procedures:</b>', '<b>Interview Process:</b>', '<b>Key words to search for the candidate</b>', '<b>LINKS</b>',
    '<b>Learn more by visiting:</b>', '<b>Link to apply:</b>', '<b>Mid-level:</b>',
    '<b>OPM Qualification Standards for the GS-1530 series can be found at the following website:</b>', '<b>Onboarding duration –</b>', '<b>Our recruitment process:</b>',
    '<b>Persons with Disabilities:</b>', '<b>Please be prepared to answer the following questions before the interview:</b>', '<b>Please submit the following documents:</b>',
    '<b>Posting Contact Email</b>', '<b>Posting Contact Name</b>', '<b>Pre-requisite checklist/expectations from the candidate before sharing the relevant profile:</b>',
    '<b>Questions that need answered</b>', '<b>Referral</b>', '<b>Required Documents:</b>', '<b>STEP 1:</b>', '<b>STEP 2:</b>', '<b>STEP 3:</b>',
    '<b>Screening of Applications Begins:</b>', '<b>Senior-level:</b>', '<b>To Know More:</b>', '<b>To all recruitment agencies:</b>', '<b>Veterans:</b>',
    '<b>Want more info?</b>', "<b>Want to learn more about Berkeley Lab's Culture, Benefits and answers to FAQs? Please visit:</b>",
    '<b>What Else You Need to Know</b>', '<b>What happens after you apply?</b>', '<b>What is our hiring process?</b>', '<b>What to expect in the hiring process:</b>',
    '<b>Work Authorization/Sponsorship</b>', '<div>Applicant Notes:</div>', '<div>Required application materials (preferably in PDF Format):</div>', '<h1>How We Interview</h1>',
    '<h1>Interview Preparation</h1>', '<h1>Interview Tips</h1>', '<h1>Our Interview Process</h1>', '<h2 class="css-14vqcyj e1tiznh50">Hiring Insights</h2>',
    '<h2 class="js-match-insights-provider-14dlqhn e1tiznh50">Profile insights</h2>', '<h2>Interview Process Overview</h2>', '<h2>Interview Stages</h2>',
    '<h3 class="jobSectionHeader" pos="H-IP">Career roadmap after the bootcamp?:</h3>', '<h3 class="jobSectionHeader" pos="H-IP">Guides:</h3>',
    '<h3 class="jobSectionHeader" pos="H-IP">Interview Procedures:</h3>', '<h3 class="jobSectionHeader">Interview Procedures:</h3>',
    '<h3 class="js-match-insights-provider-11n8e9a e1tiznh50">Licenses</h3>', '<h3 class="text-white font-semibold text-lg mb-2">How is this Skills Bootcamp delivered?</h3>',
    '<h3 class="text-white font-semibold text-lg mb-2">Programme schedule</h3>',
    '<h3 class="text-white font-semibold text-lg mb-2">What you\'ll learn in this Skills Bootcamp</h3>', '<h3>Interview Availability</h3>', '<h3>Interview Schedule</h3>',
    '<h3>Interview Stages</h3>', '<h3>What to Expect</h3>', '<h4>Company Insights</h4>', '<h4>Our Interview Process</h4>', '<li>Cover letter including:</li>',
    '<li>Experience - multiple levels can be accommodated:</li>', '<li>Please read the following guidance.</li>', '<p>2. Other supporting documents:</p>',
    '<p>Application Question(s):</p>', '<p>COVID-19 Precaution(s):</p>', '<p>Experience level:</p>', '<p>For full consideration, please submit the following document(s):</p>',
    '<p>HOW TO APPLY</p>', "<p>Here's what you can expect from our hiring process:</p>", '<p>The selection process consists of four steps:</p>',
    '<p>We will be conducting interviews on a rolling basis, so we encourage early applications.</p>', "<p>We'd love to talk to you! Please submit the following to apply:</p>",
    '<p>What to Expect Next</p>', '<span style="font-size:11.0pt">If you’re interested, please provide below details:</span>',
    '<span style="font-size:12.0pt">Interview Process</span>', '<strong>Interview process:</strong>', '<strong>Next steps:</strong>',
    '<strong>To apply:</strong>', '<strong>Want to learn more?</strong>', "<strong>What's Next:</strong>", 'Additional Information for Candidates:', 'Application Notes:',
    'Application response rate:', 'COVID-19 Considerations:', 'Date and times available to interview:', 'Find out more:',
    'For more relevant job opportunities please visit our website:', 'Full Name:', 'HIRING SPECIFICS', 'Important Information for Candidates', 'Report job',
    'Some things we think you should know', 'Stay connected',
    'The scored occupational questionnaire will evaluate you on the following competencies; please do not provide a separate written response:',
    'Visa Sponsorship Information:', 'Work Arrangement', 'You will be rated on the following Competencies as part of the assessment questionnaire for this position:',
    "<li>I'll review your application and reach out for a brief chat.</li>", "<li>I'll schedule a more in-depth interview.</li>",
    "<p>We're excited to hear from you! Kindly submit the following:</p>", "'s Interview Culture</h5>"
]

In [14]:

IS_HEADER_STR = 'True' if min_recall_pos_symbol.startswith('H-') else 'False'
PROPERTY_NAME_STR = ''
if min_recall_pos_symbol.endswith('-JT'):
    PROPERTY_NAME_STR = 'is_job_title'
elif min_recall_pos_symbol.endswith('-CS'):
    PROPERTY_NAME_STR = 'is_corporate_scope'
elif min_recall_pos_symbol.endswith('-TS'):
    PROPERTY_NAME_STR = 'is_task_scope'
elif min_recall_pos_symbol.endswith('-ER'):
    PROPERTY_NAME_STR = 'is_educational_requirement'
elif min_recall_pos_symbol.endswith('-RQ'):
    PROPERTY_NAME_STR = 'is_minimum_qualification'
elif min_recall_pos_symbol.endswith('-PQ'):
    PROPERTY_NAME_STR = 'is_preferred_qualification'
elif min_recall_pos_symbol.endswith('-SP'):
    PROPERTY_NAME_STR = 'is_supplemental_pay'
elif min_recall_pos_symbol.endswith('-OL'):
    PROPERTY_NAME_STR = 'is_office_location'
elif min_recall_pos_symbol.endswith('-JD'):
    PROPERTY_NAME_STR = 'is_job_duration'
elif min_recall_pos_symbol.endswith('-IP'):
    PROPERTY_NAME_STR = 'is_interview_procedure'
elif min_recall_pos_symbol.endswith('-LN'):
    PROPERTY_NAME_STR = 'is_legal_notification'
elif min_recall_pos_symbol.endswith('-O'):
    PROPERTY_NAME_STR = 'is_other'
elif min_recall_pos_symbol.endswith('-PD'):
    PROPERTY_NAME_STR = 'is_posting_date'

In [15]:

# Take the hand-picked HTML strings and remove their bad relationships to the database
for child_str in child_strs_list:
    def remove_relationships(tx, navigable_parent, pos_symbol, verbose=True):
        cypher_str = '''
            // Remove the SUMMARIZES relationships that are not $pos_symbol
            MATCH (pos:PartsOfSpeech)-[r:SUMMARIZES]->(np:NavigableParents {navigable_parent: $navigable_parent})
            WHERE NOT (pos.pos_symbol = $pos_symbol)
            DELETE r;'''
        if verbose:
            clear_output(wait=True)
            print(cypher_str.replace('$navigable_parent', f'"{navigable_parent}"').replace('$pos_symbol', f'"{pos_symbol}"'))
        tx.run(query=cypher_str, parameters={'navigable_parent': navigable_parent, 'pos_symbol': pos_symbol})
    with cu.driver.session() as session:
        session.write_transaction(remove_relationships, navigable_parent=child_str, pos_symbol=min_recall_pos_symbol, verbose=True)


            // Remove the SUMMARIZES relationships that are not "H-IP"
            MATCH (pos:PartsOfSpeech)-[r:SUMMARIZES]->(np:NavigableParents {navigable_parent: "'s Interview Culture</h5>"})
            WHERE NOT (pos.pos_symbol = "H-IP")
            DELETE r;


In [16]:

# Take the hand-picked HTML strings and add their characteristics to the database
for child_str in child_strs_list:
    def create_characteristics(tx, navigable_parent, verbose=True):
        cypher_str = f'''
            // Set the characteristics of the navigable_parent
            MERGE (np:NavigableParents {{navigable_parent: $navigable_parent}})
            SET
                np.is_header = {str(min_recall_pos_symbol.startswith('H-')).lower()},
                np.is_job_title = {str(min_recall_pos_symbol.endswith('-JT')).lower()},
                np.is_corporate_scope = {str(min_recall_pos_symbol.endswith('-CS')).lower()},
                np.is_task_scope = {str(min_recall_pos_symbol.endswith('-TS')).lower()},
                np.is_educational_requirement = {str(min_recall_pos_symbol.endswith('-ER')).lower()},
                np.is_minimum_qualification = {str(min_recall_pos_symbol.endswith('-RQ')).lower()},
                np.is_preferred_qualification = {str(min_recall_pos_symbol.endswith('-PQ')).lower()},
                np.is_supplemental_pay = {str(min_recall_pos_symbol.endswith('-SP')).lower()},
                np.is_office_location = {str(min_recall_pos_symbol.endswith('-OL')).lower()},
                np.is_job_duration = {str(min_recall_pos_symbol.endswith('-JD')).lower()},
                np.is_interview_procedure = {str(min_recall_pos_symbol.endswith('-IP')).lower()},
                np.is_legal_notification = {str(min_recall_pos_symbol.endswith('-LN')).lower()},
                np.is_other = {str(min_recall_pos_symbol.endswith('-O')).lower()},
                np.is_posting_date = {str(min_recall_pos_symbol.endswith('-PD')).lower()}
            RETURN
                np.navigable_parent AS navigable_parent,
                np.{PROPERTY_NAME_STR} AS {PROPERTY_NAME_STR},
                np.is_header AS is_np_header;'''
        if verbose:
            clear_output(wait=True)
            print(cypher_str.replace('$navigable_parent', f'"{navigable_parent}"'))
        return [dict(record.items()) for record in tx.run(query=cypher_str, parameters={'navigable_parent': navigable_parent})]
    row_objs_list = []
    with cu.driver.session() as session:
        row_objs_list = session.write_transaction(create_characteristics, navigable_parent=child_str)
    if row_objs_list:
        df = DataFrame(row_objs_list)
        if df.shape[0]:
            display(df.head(5))


            // Set the characteristics of the navigable_parent
            MERGE (np:NavigableParents {navigable_parent: "'s Interview Culture</h5>"})
            SET
                np.is_header = true,
                np.is_job_title = false,
                np.is_corporate_scope = false,
                np.is_task_scope = false,
                np.is_educational_requirement = false,
                np.is_minimum_qualification = false,
                np.is_preferred_qualification = false,
                np.is_supplemental_pay = false,
                np.is_office_location = false,
                np.is_job_duration = false,
                np.is_interview_procedure = true,
                np.is_legal_notification = false,
                np.is_other = false,
                np.is_posting_date = false
            RETURN
                np.navigable_parent AS navigable_parent,
                np.is_interview_procedure AS is_interview_procedure,
                np.is_header AS is_np_

,navigable_parent,is_interview_procedure,is_np_header
0,'s Interview Culture</h5>,True,True


In [17]:

# Take the hand-picked HTML strings and add the correct relationships
for child_str in child_strs_list:
    def create_relationship(tx, navigable_parent, pos_symbol, verbose=True):
        cypher_str = f'''
            // Create a SUMMARIZES relationship if it doesn't already exist
            MATCH (pos:PartsOfSpeech {{pos_symbol: $pos_symbol}})
            MATCH (np:NavigableParents {{navigable_parent: $navigable_parent}})
            MERGE (pos)-[r:SUMMARIZES]->(np)
            RETURN
                pos.pos_explanation AS pos_explanation,
                pos.pos_symbol AS pos_symbol,
                pos.{PROPERTY_NAME_STR} AS pos_{PROPERTY_NAME_STR},
                pos.is_header AS pos_is_header,
                type(r) AS relationship_type,
                np.navigable_parent AS navigable_parent,
                np.{PROPERTY_NAME_STR} AS np_{PROPERTY_NAME_STR},
                np.is_header AS np_is_header;'''
        if verbose:
            clear_output(wait=True)
            print(cypher_str.replace('$navigable_parent', f'"{navigable_parent}"').replace('$pos_symbol', f'"{pos_symbol}"'))
        results_list = tx.run(query=cypher_str, parameters={'navigable_parent': navigable_parent, 'pos_symbol': pos_symbol})
        
        return [dict(record.items()) for record in results_list]
    row_objs_list = []
    with cu.driver.session() as session:
        row_objs_list = session.write_transaction(create_relationship, navigable_parent=child_str, pos_symbol=min_recall_pos_symbol, verbose=True)
    if row_objs_list:
        df = DataFrame(row_objs_list)
        if df.shape[0]:
            display(df.head(5).T)


            // Create a SUMMARIZES relationship if it doesn't already exist
            MATCH (pos:PartsOfSpeech {pos_symbol: "H-IP"})
            MATCH (np:NavigableParents {navigable_parent: "'s Interview Culture</h5>"})
            MERGE (pos)-[r:SUMMARIZES]->(np)
            RETURN
                pos.pos_explanation AS pos_explanation,
                pos.pos_symbol AS pos_symbol,
                pos.is_interview_procedure AS pos_is_interview_procedure,
                pos.is_header AS pos_is_header,
                type(r) AS relationship_type,
                np.navigable_parent AS navigable_parent,
                np.is_interview_procedure AS np_is_interview_procedure,
                np.is_header AS np_is_header;


,0
pos_explanation,Interview Procedures Header
pos_symbol,H-IP
pos_is_interview_procedure,True
pos_is_header,True
relationship_type,SUMMARIZES
navigable_parent,'s Interview Culture</h5>
np_is_interview_procedure,True
np_is_header,True



----

# Speed/Accuracy Tradeoffs and Synthetic Dataset Generation

## Speed vs. Accuracy Tradeoffs
- To run the job-hunting pipeline efficiently on a laptop, I had to make **speed vs. accuracy tradeoffs**.
- For predicting the category of each `navigable_parent` (HTML navigable text wrapped in its parent tag), I chose **Logistic Regression**:
  - Logistic Regression is relatively fast and interpretable.
  - It sacrifices some accuracy compared to more complex machine learning models like neural networks.


## Training Data Imbalance Challenge
- The training data is highly **imbalanced**. For example, when training on **Post Date Headers (H-PD)**:
  - I have only **5 examples** of the target class.
  - There are **48,123 labeled parts of speech of non-examples**.
- This imbalance makes it difficult for the model to learn meaningful patterns for the specific class (H-PD).
- As a result, the model may perform poorly when classifying H-PD examples.

## Synthetic Dataset Generation with LLMs
- To address the training data imbalance, I am generating a **synthetic dataset** using **Large Language Models (LLMs)**:
  - I provide the few available examples as input to the LLM.
  - I refine the generation process using:
    - **Prompt engineering** before generation.
    - **Regular expressions** to clean and validate the generated examples post-generation.
    - Letting the **Logistic Regression** model pick out the best of the generated examples.
- The LLM generates additional synthetic data that resembles the target class (H-PD) closely.


## Speed and LLM Implementation Issues
- The synthetic dataset generation process is implemented using **Cohere's API**:
  - I haven't exceeded my current quota.
  - The offline open source models I have on my laptop are too slow, preventing fast turnaround.
- Manually fixing some labels in the training dataset forces me to retrain the POS Classifiers, which takes about 2 hours.

## Goals and Future Testing
- By combining **Logistic Regression** with **synthetic data augmentation**, I aim to:
  - Strike a balance between **speed**, **accuracy**, and **practicality**.
  - Improve the overall performance of the machine learning pipeline.
- I am assuming adding labeled synthetic `navigable_parent`s significantly enhances model performance without compromising efficiency. I will test when I can find some more examples in the wild.

In [ ]:

# Get all at-least-partially tagged file names
cypher_str = f'''
    // Get the tagged node counts for each file
    MATCH (pos:PartsOfSpeech)-[r1:SUMMARIZES]->(np1:NavigableParents)-[r2:NEXT]->(np2:NavigableParents)
    WITH
        r2.file_name AS file_name,
        COUNT(r1) AS tagged_count,
        COUNT(r2) AS edge_count,
        COUNT(np1) AS np_count
    RETURN np_count, tagged_count, edge_count, file_name
    ORDER BY edge_count DESC;'''
row_objs_list = []
with cu.driver.session() as session: row_objs_list = session.write_transaction(cu.do_cypher_tx, cypher_str)
if row_objs_list:
    tagged_node_counts_df = DataFrame(row_objs_list)
    if tagged_node_counts_df.shape[0]:
        display(tagged_node_counts_df.sample(5))

In [ ]:

# Get all POS symbols for the at-least-partially tagged files
filenames_list = tagged_node_counts_df.file_name.tolist()
filenames_str = '", "'.join(filenames_list)
cypher_str = f'''
    // Get child string and POS for each at-least-partially tagged file
    MATCH (pos:PartsOfSpeech)-[r1:SUMMARIZES]->(np1:NavigableParents)-[r2:NEXT]->(np2:NavigableParents)
    WHERE
        r2.file_name IN ["{filenames_str}"]
    RETURN
        np1.navigable_parent AS text,
        pos.pos_symbol AS pos_symbol;'''
# print(cypher_str)
row_objs_list = []
with cu.driver.session() as session: row_objs_list = session.write_transaction(cu.do_cypher_tx, cypher_str)
if row_objs_list:
    file_tags_df = DataFrame(row_objs_list)
    if file_tags_df.shape[0]:
        display(file_tags_df.sample(5))

In [ ]:

min_support_pos_symbol = file_tags_df.groupby('pos_symbol').count().reset_index().rename(columns={'text': 'labeled_count'}).sort_values('labeled_count').iloc[0].pos_symbol
print(f'The training data is highly imbalanced. For example, when training on {hc.POS_EXPLANATION_DICT[min_recall_pos_symbol]} ({min_recall_pos_symbol}):')
print(f'    I have only {target_class_df.shape[0]:,} examples of the target class.')
print(f'    There are {file_tags_df.shape[0]:,} labeled parts of speech of non-examples.')

In [ ]:

# Get all at-least-partially tagged file names
cypher_str = '''
    // Remove the SUMMARIZES relationships that are not "O-PD"
    MATCH (pos:PartsOfSpeech)-[r1:SUMMARIZES]->(np1:NavigableParents)
    WHERE
        np1.navigable_parent IN ['<span class="jobsearch-HiringInsights-entry--text">Posted 7 days ago</span>']
        AND NOT (pos.pos_symbol = "O-PD")
    DELETE r1;'''
pyperclip.copy(cypher_str)